In [1]:
import math
import os
import pandas as pd
import subprocess
from pathlib import Path

## Setup

In [16]:
# experiment setup
my_pdk = 'sky130'
my_design = 'sram_sim'
my_inst = '/sram_sim/mem0'
my_clock_period = 10 # ns

# useful paths
e2e_dpath = (Path(os.getcwd()).parent).parent
tests_dpath = e2e_dpath/f'experiments/tests-{my_pdk}'

## Generate Experiment Files 

### Input Files

In [18]:
def parse_sram_cfg(config):
    parts = config.split("_")[1]
    num_words = int(parts.split("x")[0])
    data_width = int(parts.split("x")[1].split("m")[0])
    write_size = int(parts.split("w")[1])

    addr_width = int(math.ceil(math.log2(num_words)))
    wmask_width = data_width // write_size

    return {
        "SRAM"        : config,
        "DATA_WIDTH"  : data_width,
        "ADDR_WIDTH"  : addr_width,
        "WMASK_WIDTH" : wmask_width
    }

In [19]:

N_ITER = 50

read, write = 0, 1

zero_din = 0
max_din32 = (1 << 32) - 1

zero_addr, one_addr = 0, 1
max_addr64 = (1 << 6) - 1

zero_wmask = 0
max_wmask32w8 = (1 << 4) - 1

# tests_dict
#   <sram_configuration>-<test_name>:
#       inputs  : List
#                 Each item is a list of items per line to be generated in input.txt
#       defines : Dict  
#                 Each key-value pair is a parameter=value appended to the Hammer config
tests_dict = {
    'sram64x32-zero': {
        'inputs': [(read, zero_din, zero_addr, zero_wmask) for _ in range(N_ITER)],
        'defines': parse_sram_cfg("sram22_64x32m4w8"),
    },
    'sram64x32-max_input_switching': {
        'inputs': [(read, zero_din, zero_addr, zero_wmask) if row % 2 else (write, max_din32, max_addr64, max_wmask32w8) for row in range(N_ITER)],
        'defines': parse_sram_cfg("sram22_64x32m4w8"),
    },
    'sram64x32-max_output_switching': {
        'inputs': [(write, zero_din, zero_addr, max_wmask32w8), (write, max_din32, one_addr, max_wmask32w8)] + [(read, zero_din, zero_addr, zero_wmask) if row % 2 else (read, zero_din, one_addr, zero_wmask) for row in range(N_ITER)],
        'defines': parse_sram_cfg("sram22_64x32m4w8"),
    },

    # 'sram128x16-zero': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_128x16m4w8"),
    # },
    # 'sram128x16-max_input_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_128x16m4w8"),
    # },
    # 'sram128x16-max_output_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_128x16m4w8"),
    # },

    # 'sram256x64-zero': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_256x64m4w8"),
    # },
    # 'sram256x64-max_input_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_256x64m4w8"),
    # },
    # 'sram256x64-max_output_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_256x64m4w8"),
    # },

    # 'sram1024x32-zero': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_1024x32m4w8"),
    # },
    # 'sram1024x32-max_input_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_1024x32m4w8"),
    # },
    # 'sram1024x32-max_output_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_1024x32m4w8"),
    # },

    # 'sram2048x8-zero': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_2048x8m4w8"),
    # },
    # 'sram2048x8-max_input_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_2048x8m4w8"),
    # },
    # 'sram2048x8-max_output_switching': {
    #     'inputs': [],
    #     'defines': parse_sram_cfg("sram22_2048x8m4w8"),
    # }
}

In [20]:
# create dirs
for t in tests_dict:
    # directory for all input/output files
    root = tests_dpath/t
    root.mkdir(exist_ok=True,parents=True)
    tests_dict[t]['defines']['TESTROOT'] = root
    tests_dict[t]['root'] = root
    # hammer build directory
    design = t.split('-')[0]
    obj_dir = f"build-{my_pdk}-cm/{design}"
    tests_dict[t]['design'] = design
    tests_dict[t]['obj_dir'] = obj_dir
    tests_dict[t]['obj_dpath'] = e2e_dpath/obj_dir
    

# write out input.txt
for t in tests_dict:
    with (tests_dict[t]['root']/'input.txt').open('w') as f:
        for i in tests_dict[t]['inputs']:
            f.write(" ".join(['{0:b}'.format(ii) for ii in i]) + '\n')

### Hammer Config

In [21]:
def write_cfg_rtl(test_dict):
  defines_str = '\n'.join( [ f"  - {key}={val}" for key,val in test_dict['defines'].items() ] )
  cfg = f"""\
design.defines: &DEFINES
  - CLOCK_PERIOD=10
{defines_str}

sim.inputs:
  defines: *DEFINES
  defines_meta: 'append'

synthesis.inputs.defines: *DEFINES

power.inputs.defines: *DEFINES

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: rtl
  input_files: [src/{my_design}.v]
  report_configs:
    - waveform_path: {test_dict['root']}/output.fsdb
      report_stem: {test_dict['root']}/power
      toggle_signal: clock
      num_toggles: 1
      levels: 2
      inst: {my_inst}
      output_formats:
      - report
      - plot_profile
"""
  with (test_dict['root']/'config.yml').open('w') as f:
    f.write(cfg)

for t in tests_dict:
    write_cfg_rtl(tests_dict[t])

## Run Experiments

In [22]:
# generate custom make str for each test
for t in tests_dict:
    cfg = str(tests_dict[t]['root']/'config.yml')
    tests_dict[t]['make'] = f"design={my_design} OBJ_DIR={tests_dict[t]['obj_dir']} extra={cfg}"

# build
build_dirs = {tests_dict[t]['obj_dir']: t for t in tests_dict} # run build once per build dir (not once per test)
for bd,t in build_dirs.items():
    command = f"make build -B {tests_dict[t]['make']} -B"
    print(f"Running: {command}")
    fpath = tests_dict[t]['root']/'power.hier.power.rpt'
    if not fpath.exists():
        subprocess.run(command, shell=True, cwd=e2e_dpath, check=True)
print()

# sim-rtl
for t in tests_dict:
    command = f"make redo-sim-rtl -B {tests_dict[t]['make']}"
    print(f"Running: {command}")
    fpath = tests_dict[t]['root']/'power.hier.power.rpt'
    if not fpath.exists():
        subprocess.run(command, shell=True, cwd=e2e_dpath, check=True)
print()

# power-rtl
for t in tests_dict:
    # re-use pre_report_power database if it's already generated (i.e. skip synthesis)
    make_target = "redo-power-rtl args='--only_step report_power'" \
            if (tests_dict[t]['obj_dpath']/'power-rtl-rundir/pre_report_power').exists() else 'power-rtl'
    command = f"make {make_target} -B {tests_dict[t]['make']}"
    print(f"Running: {command}")
    fpath = tests_dict[t]['root']/'power.hier.power.rpt'
    if not fpath.exists():
        subprocess.run(command, shell=True, cwd=e2e_dpath, check=True)
print()

Running: make build -B design=sram_sim OBJ_DIR=build-sky130-cm/sram64x32 extra=/bwrcq/scratch/henrycen/main/hammer/e2e/experiments/tests-sky130/sram64x32-max_output_switching/config.yml -B

Running: make redo-sim-rtl -B design=sram_sim OBJ_DIR=build-sky130-cm/sram64x32 extra=/bwrcq/scratch/henrycen/main/hammer/e2e/experiments/tests-sky130/sram64x32-zero/config.yml
Running: make redo-sim-rtl -B design=sram_sim OBJ_DIR=build-sky130-cm/sram64x32 extra=/bwrcq/scratch/henrycen/main/hammer/e2e/experiments/tests-sky130/sram64x32-max_input_switching/config.yml
Running: make redo-sim-rtl -B design=sram_sim OBJ_DIR=build-sky130-cm/sram64x32 extra=/bwrcq/scratch/henrycen/main/hammer/e2e/experiments/tests-sky130/sram64x32-max_output_switching/config.yml

Running: make redo-power-rtl args='--only_step report_power' -B design=sram_sim OBJ_DIR=build-sky130-cm/sram64x32 extra=/bwrcq/scratch/henrycen/main/hammer/e2e/experiments/tests-sky130/sram64x32-zero/config.yml
Running: make redo-power-rtl args='-

## Parse Results

In [23]:
def parse_hier_power_rpt(fpath,inst) -> list:
    with fpath.open('r') as f: lines = f.readlines()
    for l in lines:
        words = l.split()
        if inst == words[-1]:
            print(words[2:6])
            return [float(p) for p in words[0:-1]]
    return []

power = list([])
for t in tests_dict:
    fpath = tests_dict[t]['root']/'power.hier.power.rpt'
    power.append(parse_hier_power_rpt(fpath,my_inst))

power = pd.DataFrame(power,   #  mW
                     columns=['Leakage','Internal','Switching','Total'],
                     index=tests_dict.keys()) # type: ignore

energy = power * my_clock_period # mW * ns = pJ
energy

['4.67247e-05', '4.86227e-01', '/sram_sim/mem0']
['6.05415e-04', '4.53329e-01', '/sram_sim/mem0']
['4.08154e-04', '4.63063e-01', '/sram_sim/mem0']


,Leakage,Internal,Switching,Total
sram64x32-zero,0.700542,4.16126,0.000467,4.86227
sram64x32-max_input_switching,0.700542,3.82669,0.006054,4.53329
sram64x32-max_output_switching,0.700542,3.92601,0.004082,4.63063
